©2020-2021 ETH Zurich, Pagan Nicolò; D-ITET; Automatic Control Lab

This program is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

This program is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with this program.  If not, see <https://www.gnu.org/licenses/>.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
import tikzplotlib as tikz
from scipy import stats
from scipy.stats import ks_2samp

plt.rcParams['figure.figsize'] = [10,7]
random.seed(0)

In [ ]:
# The function returns the unique element of a list.
def unique(list1): 
    x = np.array(list1) 
    x = (np.unique(x))
    return x.tolist()

# From the file containing the edges list, the function imports the list of edges and finds the set of nodes.
def import_nodes(filename):
    edges = pd.read_csv(filename)
    nodes = edges['Source'].to_list()
    nodes = [nodes, edges['Target'].to_list()]
    nodes = unique(nodes)
    return nodes, edges

# From the list of edges and nodes, the function creates a dataframe in which, each row is associated with a node.
# The row contains the node id, its in-degree, and its ranking (based on the in-degree).
# Rows are ordered in decreasing in-degree, and increasing ranking.
def create_nodes_df(nodes, edges):
    indegree = [0]*len(nodes)
    outdegree = [0]*len(nodes)
    data = {'id':nodes, 'indegree':   indegree, 'outdegree': outdegree}
    nodes_df = pd.DataFrame(data)
    edges_list = edges.values.tolist()
    for edge in edges_list:
        nodes_df.loc[nodes_df['id']==edge[1], 'indegree'] += 1
        nodes_df.loc[nodes_df['id']==edge[0], 'outdegree'] += 1
    nodes_df.sort_values(by=['indegree'], inplace=True, ascending=False)
    
    rank_array = np.linspace(1, len(nodes), len(nodes), dtype=int)
    rank_list = rank_array.tolist()
    nodes_df['rank']=rank_list
    
    return nodes_df

In [ ]:
[nodes, edges] = import_nodes('graph.csv') # Note that the graph file must be in the same folder, or the path must be specified
nodes_df = create_nodes_df(nodes, edges)

Using the edge list, we create a data-frame (friends_df) in which, for each user id, we store the rank of the userid's followee, sequentially ordered (the first one is the oldest followee).

In [ ]:
friends_df = pd.DataFrame(columns=['id', 'friends'])
node = edges['Source'][0]
friends = []
for index, edge in edges.iterrows():
    if (node != edge['Source']):
        friends_df = friends_df.append({'id' : node , 'friends' : friends}, ignore_index=True)
        node = edge['Source']
        friends = []
    friend_rank = nodes_df.loc[nodes_df['id']==edge['Target']]
    friends.append(friend_rank['rank'].values[0]) 

# Computing violation with respect to the median
The following code computes, for each node (with at least two followees), the probability p_i (as defined in the manuscript), and it stores it in the vector p.
To do so, it uses the list of followees, ordered in temporal sequence, as given by the Twitter API.
Similarly, it computes the same probability under the null hypothesis, i.e., after reshuffling the list of followees (thus, removing any temporal pattern).

In [ ]:
random.seed(0)
p = []
p_null_hypothesis = []
for node in friends_df['id']:
    friends = (friends_df.loc[friends_df['id'] == node, 'friends'].values[0]).copy()
    friends_null_hypothesis = random.sample(friends, len(friends))
    t = np.linspace(1, len(friends), len(friends), dtype=int)
    series  = {'time': t, 'friends': friends}
    violations = np.NaN
    violations_null_hypothesis = np.NaN
    median = []
    median.append(friends[0])
    median_null_hypothesis = []
    median_null_hypothesis.append(friends_null_hypothesis[0])
    if (len(friends)>1):
        violations = 0
        violations_null_hypothesis = 0
    for i in range(1, len(friends)):
        if (friends[i]>median[-1]):
            violations+=1
        if (friends_null_hypothesis[i]>median_null_hypothesis[-1]):
            violations_null_hypothesis+=1
        median.append(np.median(friends[0:i+1]))
        median_null_hypothesis.append(np.median(friends_null_hypothesis[0:i+1]))
    if (len(friends)>1):
        p.append(violations/(len(friends)-1))
        p_null_hypothesis.append(violations_null_hypothesis/(len(friends)-1))
    else:
        p.append(np.NaN)
        p_null_hypothesis.append(np.NaN)
plt.hist(p, bins=19, alpha=0.5, label='Data')
plt.hist(p_null_hypothesis, bins=19, alpha = 0.5, label='Null Hypotesis')
plt.xlim((0,1))
plt.xlabel('Median violation probability')
plt.ylabel('frequency')
plt.title('Median rule violations')
plt.legend(loc="upper right")
tikz.save("median_violation_histogram.tikz")
print('Data.')
print('Size:', np.count_nonzero(~np.isnan(p)))
print('Mean:', np.nanmean(p))
print('Median:', np.nanmedian(p))
print('Std-dev:', np.nanstd(p))
print('')
print('0-hypotesis:')
print('Mean:', np.nanmean(p_null_hypothesis))
print('Median:', np.nanmedian(p_null_hypothesis))
print('Std-dev:', np.nanstd(p_null_hypothesis))
print('')
print('Kolmogorov-Smirnov tests, two samples')
ks_2samp(p, p_null_hypothesis)